### #BreakItDown

0. [Sequential Spreadsheet](https://docs.google.com/spreadsheets/d/e/2PACX-1vSAE2tBAnAdXsxk9a9YClFN7MSEVhzEmJD01ewwtooMLxL-Ilod26EbdD8sZeZk0ybiqD-jqT-9RZbn) should more cohesive serieses (ideal case)
1. Extract partial captions from query + edge cases
    1. Topic, difficulty, and duration
2. Form the perfect playlist-making prompt
    1. Optimize for chroma DB
    2. Learn if you really need chroma DB to avoid hallucinations
3. Learn how to exclude based on distance?
4. Work with heroku postgres data. 
    1. See current search implementation

___

### OPTIONAL
-1. Message Heather

0. Open Chase Checking/Savings
1. Deactivate BOFA Debit/Credit
2. Deactivate Discover CC

___

In [29]:
import openai
from pprint import pprint

def extract_details(user_input):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "As an education-focused assistant, your task is to extract key details from user queries to help them maximize their learning experience."
            },
            {
                "role": "system",
                "content": "Please return a string that includes the following details, separated by a tilde (~): the detailed topic (the more detailed, the better), the difficulty level, the number of videos needed to form a playlist (integer only), the name of the playlist, and a concise description of the playlist."
            },
            {
                "role": "user",
                "content": f"Please extract the following details from this query: '{user_input}'. The details are: topic, difficulty, duration, potential playlist title, and playlist description."
            },
            {
                "role": "user",
                "content": "Remember, your response should be a single string with each value separated by a tilde (~)."
            }
        ]
    )
    details = response.choices[0].message['content'].strip()
    return details

def split_details(details):
    topic, difficulty, duration, playlist_title, playlist_desc = details.split('~')
    return topic.strip(), difficulty.strip(), duration.strip(), playlist_title.strip(), playlist_desc.strip()

# Example Usage
user_input = "I want to learn setting up a Docker environment quickly."
details = extract_details(user_input)

print(details, "\n")
topic, difficulty, duration, playlist_title, playlist_desc = split_details(details)

pprint(f"Topic: {topic}, Difficulty: {difficulty}, Number of Videos: {duration}, Playlist Title: {playlist_title}, Playlist Description: {playlist_desc}")


Docker: Setting up a Docker environment efficiently~Intermediate~5~"Docker Setup Masterclass"~Comprehensive playlist covering step-by-step tutorials on setting up Docker environment quickly and effectively. 

('Topic: Docker: Setting up a Docker environment efficiently, Difficulty: '
 'Intermediate, Number of Videos: 5, Playlist Title: "Docker Setup '
 'Masterclass", Playlist Description: Comprehensive playlist covering '
 'step-by-step tutorials on setting up Docker environment quickly and '
 'effectively.')


In [31]:
def create_curriculum_prompt(topic, difficulty, duration):
    # Constructs the prompt to generate a curriculum
    return f"Create a short video curriculum on {topic} with a {difficulty} difficulty level, designed to be covered in excatly {duration} videos."

def generate_curriculum(topic, difficulty, duration):
    prompt = create_curriculum_prompt(topic, difficulty, duration)

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an experienced educational computer scientist now teaching others."},
            {"role": "system", "content": "You return only a list without sublists, following the exact number of videos as specificed by the prompt."},
            {"role": "system", "content": "You don't add prefix and suffixes, just the video title. Make sure the video title is detailed enough for a vectorstore DB to query easily."},
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message['content']

# Example Usage
curriculum = generate_curriculum(topic, difficulty, duration)
print(f"Generated Curriculum for Playlist:")
print(playlist_title, "\n", playlist_desc, "\n")
print(curriculum)


Generated Curriculum for Playlist:
"Docker Setup Masterclass" 
 Comprehensive playlist covering step-by-step tutorials on setting up Docker environment quickly and effectively. 

1. Docker Setup: Installing Docker on Windows, macOS, and Linux
2. Docker Images and Containers: Understanding the relationship and creating your first container
3. Docker Compose: Managing multi-container applications efficiently
4. Docker Volumes: Managing data persistence in Docker
5. Docker Networking: Configuring network settings for containers
